In [1]:
!bash /home/azureuser/cloudfiles/code/blobfuse/blobfuse_raadsinformatie.sh

In [2]:
import sys
sys.path.append("..")

# Select where to run notebook: "azure" or "local"
my_run = "azure"

import my_secrets as sc
import settings as st

if my_run == "azure":
    import config_azure as cf
elif my_run == "local":
    import config as cf

- Load data - just first 2 docs of training set
- text truncation
- load file with prompts


In [3]:
def text_truncation(tokens_list, maximum=500):
    selected_tokens = tokens_list[:maximum]  # Select the first 300 tokens
    return ' '.join(selected_tokens)  # Convert the list back to text

In [4]:
import pandas as pd
df = pd.read_pickle(f"{cf.output_path}/txtfiles.pkl")
print(list(set(df['label'])))
df = df.loc[df['set']=='train'].head(2)
df['text_trunc'] = df['tokens'].apply(text_truncation)
display(df)

import sys
sys.path.append('../scripts/') 
import prompt_template as pt


['Besluit', 'Raadsnotulen', 'Motie', 'Voordracht', 'Agenda', 'Raadsadres', 'Factsheets', 'Actualiteit', 'Onderzoeksrapport', 'Schriftelijke Vragen', 'Brief', 'Termijnagenda']


,label,path,id,set,text,tokens,token_count,clean_tokens,clean_tokens_count,pdf_path,num_pages,clean_text,text_trunc
0,Motie,/home/azureuser/cloudfiles/code/blobfuse/raads...,0,train,Gemeente Amsterdam\n% Gemeenteraad R\n% Gemeen...,"[Gemeente, Amsterdam, %, Gemeenteraad, R, %, G...",395,"[Gemeente, Amsterdam, Gemeenteraad, Gemeentebl...",205,/home/azureuser/cloudfiles/code/blobfuse/raads...,2.0,Gemeente Amsterdam Gemeenteraad Gemeenteblad M...,Gemeente Amsterdam % Gemeenteraad R % Gemeente...
1,Motie,/home/azureuser/cloudfiles/code/blobfuse/raads...,1,train,Gemeente Amsterdam\n\n% Gemeenteraad R\n\n% Ge...,"[Gemeente, Amsterdam, %, Gemeenteraad, R, %, G...",390,"[Gemeente, Amsterdam, Gemeenteraad, Gemeentebl...",197,/home/azureuser/cloudfiles/code/blobfuse/raads...,2.0,Gemeente Amsterdam Gemeenteraad Gemeenteblad M...,Gemeente Amsterdam % Gemeenteraad R % Gemeente...


#### Tryout GEITje
Load chatbot

In [5]:
from transformers import pipeline, Conversation

chatbot = pipeline(task='conversational', model='Rijgersberg/GEITje-7B-chat-v2',
                   device_map='auto')

2024-04-03 08:52:00.798897: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2024-04-03 08:52:00.798918: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.


Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

Simple query

In [ ]:
print(chatbot(
    Conversation('Welk woord hoort er niet in dit rijtje thuis: "auto, vliegtuig, geitje, bus"?')
))

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


Conversation id: c7f94172-c476-4fc4-a1cf-27485289b9d1
user: Welk woord hoort er niet in dit rijtje thuis: "auto, vliegtuig, geitje, bus"?
assistant: Geitje hoort er niet in thuis.



### Experiment set-up
Prompt GEITje for each document and save the prediction, return response, response time and the prompt version

In [11]:
from collections import Counter
import re

# given the string response, extract the prediction
def get_prediction_from_response(response, classes_list):
    predictions = [True if category.lower() in response.lower() else False for category in classes_list]

    # check if multiple classes were named, this is a prediction error
    if Counter(predictions)[True] > 1:
        return "PredictionError"

    # check if exactly one class is named, this is the prediction
    elif Counter(predictions)[True] == 1:
        prediction = [category.lower() for category in classes_list if category.lower() in response.lower()]
        return prediction[0]

    # if no class is named, then this is a no prediction error
    else:
        return 'NoPrediction'

# extract the promptfunction name
def get_promptfunction_name(prompt_function):
    string = f"{prompt_function}"
    match = re.search(r'<function\s+(\w+)', string)
    if match:
        function_name = match.group(1)
        return function_name
    else:
        return f"{prompt_function}"
        

In [14]:
import time

""" Given a dataframe with txt, return a df with predictions """
# docs_df = dataframe with the documents that need to be predicted
# text_column = name of the column that includes the input_text. Can be different based on the text representation method. 
# prompt_function = prompt template -> ONLY prompt templates that take classes and doc as input (ZERO SHOT)

def zero_shot_predictions_incontextlearning(docs_df, text_column, prompt_function):
    # get a list of the possible classes
    classes_list = pt.get_class_list()

    results_df = pd.DataFrame(columns = ['id', 'path', 'text_column', 'prompt_function', 'response', 'prediction', 'label', 'runtime'])
    
    # prompt each document
    for index, row in docs_df.iterrows():
        start_time = time.time()

        # get the prompt, with the classes and doc filled in
        txt = row[text_column]
        prompt = prompt_function(classes_list, txt)

        # prompt and get the response
        converse = chatbot(Conversation(prompt))
        response = converse[1]['content']

        # extract prediction from response
        prediction = get_prediction_from_response(response, classes_list)

        # save results in dataframe
        results_df.loc[len(results_df)] = {
            'id': row['id'],
            'path' : row['path'],
            'text_column' : text_column,
            'prompt_function': get_promptfunction_name(prompt_function),
            'response':response,
            'prediction':prediction,
            'label':row['label'].lower(),
            'runtime':time.time()-start_time
        }

    return results_df

""" Run a prediction function -> can be ZeroShot or FewShot """


def run_prediction(docs_df, text_column, prompt_function, learning='ZeroShot'):
    begin_time = time.time()
    if learning == 'ZeroShot':
        res = zero_shot_predictions_incontextlearning(docs_df, text_column, prompt_function)
    return res



res = run_prediction(df, 'text_trunc', pt.simple_prompt)
display(res)


/anaconda/envs/ThesisAmsterdamEnvironment19/lib/python3.9/site-packages/transformers/pipelines/base.py:1157: UserWarning: You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a dataset
  warnings.warn(
The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
/anaconda/envs/ThesisAmsterdamEnvironment19/lib/python3.9/site-packages/transformers/pipelines/base.py:1157: UserWarning: You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a dataset
  warnings.warn(
The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


,id,path,text_column,prompt_function,response,prediction,label,runtime
0,0,/home/azureuser/cloudfiles/code/blobfuse/raads...,text_trunc,simple_prompt,Brief,brief,motie,7.774881
1,1,/home/azureuser/cloudfiles/code/blobfuse/raads...,text_trunc,simple_prompt,Voordracht,voordracht,motie,11.156590
